In [1]:
import sys
import csv
import time
import copy
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn import datasets
import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from random import getrandbits
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_curve, auc
from random import randint
import random
import copy

In [2]:
def sigmoid(z):
    return 1. / (1 + np.exp(-z))

In [3]:
def sample_floats(low, high, k=1):
    """ Return a k-length list of unique random floats
        in the range of low <= x <= high
    """
    result = []
    seen = set()
    for i in range(k):
        x = random.uniform(low, high)
        while x in seen:
            x = random.uniform(low, high)
        seen.add(x)
        result.append(x)
    return result

In [4]:
def normalize(pt):
    df=pd.read_csv(str(pt) + '/4_time_line.csv', header=0, sep=',')
    df['week']=df.loc[:,"week_click":"week_undisp"].min(axis=1)
    col=[col for col in df.columns if col not in ["week_click","week_book","week_reply","week_undisp"]]
    df=df[col]
    user_list=np.unique(df.user_id)
    #split a provided dataset by taking into account the time_line 
    test=[]
    train=[]
    for i in tqdm_notebook(range(len(user_list))):
        df_split = np.array_split(df[df.user_id==user_list[i]].sort_values(by='week'), 3)
        train.append(list(df_split[0].index))
        train.append(list(df_split[1].index))
        test.append(list(df_split[2].index))
    train=list(itertools.chain(*train))
    test=list(itertools.chain(*test))
#======================================
    X_train=df[df.index.isin(train)]
    X_test=df[df.index.isin(test)]
    X_train.set_index(['user_id','item_id'], inplace=True)
    X_test.set_index(['user_id','item_id'], inplace=True)
#======================================
    Y_train_click=X_train.click
    Y_train_book=X_train.book
    Y_train_reply=X_train.reply
    
    Y_test_click=X_test.click
    Y_test_book=X_test.book
    Y_test_reply=X_test.reply
#======================================    
    col=[col for col in df.columns if col not in ['displayed','user_id','item_id',"click","book","reply","week"]]
    X_train=X_train[col]
    X_test=X_test[col]
    
    X_train = X_train.astype(np.float64)
    X_test = X_test.astype(np.float64)
    #min_max normalization just for contextual features
    min_max_scaler = preprocessing.MinMaxScaler()
    X_train_minmax = min_max_scaler.fit_transform(X_train.iloc[:,15:])
    X_test_minmax = min_max_scaler.transform(X_test.iloc[:,15:])

    X_train_minmax = pd.DataFrame(X_train_minmax)
    X_test_minmax = pd.DataFrame(X_test_minmax)
    X_train_minmax.columns = X_train.iloc[:,15:].columns
    X_test_minmax.columns = X_test.iloc[:,15:].columns
    X_train_minmax.index = X_train.iloc[:,15:].index
    X_test_minmax.index = X_test.iloc[:,15:].index
    #standardization just for contextual features
    sc=StandardScaler()
    sc.fit(X_train_minmax)
    X_train_std=sc.transform(X_train_minmax)
    X_test_std=sc.transform(X_test_minmax)

    X_train_std = pd.DataFrame(X_train_std)
    X_test_std = pd.DataFrame(X_test_std)
    X_train_std.columns = X_train.iloc[:,15:].columns
    X_test_std.columns = X_test.iloc[:,15:].columns
    X_train_std.index = X_train.iloc[:,15:].index
    X_test_std.index = X_test.iloc[:,15:].index
#=======================================
    U=np.unique(X_train_std.reset_index().user_id)
    I=np.unique(X_train_std.reset_index().item_id)
    Y_train_click=Y_train_click.reset_index()
    Y_train_book=Y_train_book.reset_index()
    Y_train_reply=Y_train_reply.reset_index()
    X_train_std=X_train_std.reset_index()
    X_test_std=X_test_std.reset_index()
    Y_test_click=Y_test_click.reset_index()
    Y_test_book=Y_test_book.reset_index()
    Y_test_reply=Y_test_reply.reset_index()
#=========================Convert training and test sets from pandas dataframe to dictionaries
    X_train_dict={}
    for i in tqdm_notebook(xrange(len(X_train_std))):
        X_train_dict[X_train_std.user_id[i],X_train_std.item_id[i]]=X_train.iloc[i,:15].values.tolist()+\
        X_train_std.iloc[i,2:].values.tolist()
    X_test_dict={}
    for i in tqdm_notebook(xrange(len(X_test_std))):
            X_test_dict[X_test_std.user_id[i],X_test_std.item_id[i]]=[1]+X_test.iloc[i,:15].values.tolist()+\
            X_test_std.iloc[i,2:].values.tolist()

   
    Y_train_dict_click={}
    Y_train_dict_book={}
    Y_train_dict_reply={}
    for i in tqdm_notebook(xrange(len(Y_train_click))):
        Y_train_dict_click[Y_train_click.user_id[i],Y_train_click.item_id[i]]=Y_train_click.iloc[i,2]
        Y_train_dict_book[Y_train_book.user_id[i],Y_train_book.item_id[i]]=Y_train_book.iloc[i,2]
        Y_train_dict_reply[Y_train_reply.user_id[i],Y_train_reply.item_id[i]]=Y_train_reply.iloc[i,2]
#======Create assistance dictionary where for eash user were assgined all offers  which were interacted by target user   
    indx_dict={}
    for i in range(len(U)):
        indx_dict[U[i]]=X_train_std[X_train_std.user_id==U[i]].iloc[:,:2].values.tolist()
#========In Stochastic Gradient Descent which we will use in further, the loss function defined as logistic loss function for Y\in{-1,+1}   
   
    for key,values in Y_train_dict_click.iteritems():
        if values == 0:
            Y_train_dict_click[key] = -1
    for key,values in Y_train_dict_book.iteritems():
        if values == 0:
            Y_train_dict_book[key] = -1
    for key,values in Y_train_dict_reply.iteritems():
        if values == 0:
            Y_train_dict_reply[key] = -1
            
#=============Embedding representation for users and items

    U_repr = {U[i]:[v for v in sample_floats(0,1,k=X_train.shape[1]-1)] for i in range (len(U))}
    I_repr= {I[j]:[w for w in sample_floats(0,1,k=X_train.shape[1]-1)] for j in range (len(I))}

#============Single-task learning for clicks task
    
    #Inputs:
    eta=1e-6
    lamb=1e-2
    nb_runs = 600000
    eps=1e-3
    #Initialization:
    w1=[w for w in sample_floats(0,1,k=32)]
    global_loss = [1,0]
    nb=1
    while abs(global_loss[nb-1]-global_loss[nb])>eps:
        loss_local=0
        for j in tqdm_notebook(range(nb_runs)):
            u = randint(0,len(U)-1)

            target_samples=indx_dict[U[u]]
            target_lables=[Y_train_dict_click[tuple(elem)] for elem in target_samples]
            positive=[target_samples[m] for m in [l for l, e in enumerate(target_lables) if e == 1]]
            negative=[target_samples[m] for m in [l for l, e in enumerate(target_lables) if e == -1]]

            if (len(positive)!=0 and len(negative)!=0): 

                p=randint(0, len(positive)-1)
                n=randint(0, len(negative)-1)


                item_plus = X_train_dict[tuple(positive[p])]
                item_minus = X_train_dict[tuple(negative[n])] 

                diff = (  np.array(item_minus) - np.array(item_plus)  ).tolist()
                a = not getrandbits(1)
                if a==True:
                    l_1 = np.log( 1. + np.exp( np.dot([1.]+diff , w1 ) ) )
                    dl_1=( [1.] + diff ) / (1. + np.exp( (-1.) * np.dot( [1.] + diff ,w1 ) ) )
                else:
                    l_1 = np.log(1.+ np.exp( np.dot( [-1.] + diff , w1 ) ) )
                    dl_1=( [-1.] + diff ) / (1. + np.exp( (-1.) * np.dot( [-1.] + diff ,w1) ) )

                w1 = w1 - eta * ( (dl_1) + (2 * lamb * np.sum( w1 ) ) )

                loss_local = loss_local + l_1 + ( lamb * np.linalg.norm( w1,2 )**2 )


        global_loss.append(loss_local/nb_runs)
        nb+=1   
        
#============Single-task learning for bookmarks task

    #Inputs:
    eta=1e-6
    lamb=1e-1
    eps=1e-3
    nb_runs = 600000
    #Initialization:
    w2=[w for w in sample_floats(0,1,k=32)]
    global_loss=[0,1]
    nb=1
    while abs(global_loss[nb-1]-global_loss[nb])>eps:
        loss_local=0
        for j in tqdm_notebook(range(nb_runs)):
            u = randint(0,len(U)-1)

            target_samples=indx_dict[U[u]]
            target_lables=[Y_train_dict_click[tuple(elem)] for elem in target_samples]
            positive=[target_samples[m] for m in [l for l, e in enumerate(target_lables) if e == 1]]
            negative=[target_samples[m] for m in [l for l, e in enumerate(target_lables) if e == -1]]

            if (len(positive)!=0 and len(negative)!=0): 

                p=randint(0, len(positive)-1)
                n=randint(0, len(negative)-1)

                book_plus_label = Y_train_dict_book[tuple(positive[p])]
                book_minus_label = Y_train_dict_book[tuple(negative[n])]
                reply_plus_label = Y_train_dict_reply[tuple(positive[p])]
                reply_minus_label = Y_train_dict_reply[tuple(negative[n])]

                item_plus = X_train_dict[tuple(positive[p])]
                item_minus = X_train_dict[tuple(negative[n])] 

                diff = ( np.array(item_minus) - np.array(item_plus) ).tolist()            

                l_2_plus = (1./2) * np.log( 1 + ( 1. / np.exp( book_plus_label * np.dot([1.] + item_plus , w2 )  ) ) ) 
                l_2_minus = (1./2) * np.log( 1 + ( 1. / np.exp( book_minus_label * np.dot([1.] + item_minus , w2 ) ) ) ) 

                dl_2_plus = ( 1./2) * ( ( (-1.)* book_plus_label * np.array([1.] + item_plus) ) / ( 1. + np.exp( book_plus_label * np.dot( [1.] + item_plus , w2 ) ) ) ) 
                dl_2_minus = ( 1./2) * ( ( (-1.)* book_minus_label * np.array([1.] + item_minus) ) / ( 1. + np.exp( book_minus_label * np.dot( [1.] + item_minus , w2 ) ) ) ) 

                if book_plus_label == 1 :
                    dl_2_plus = 7 * dl_2_plus
                if book_minus_label == 1 :
                    dl_2_minus = 7 * dl_2_minus


                w2 = w2 - eta * ( (dl_2_plus + dl_2_minus) + (2 * lamb * np.sum( w2 ) ) )

                loss_local = loss_local + l_2_plus + l_2_minus + ( lamb * np.linalg.norm( w2,2 )**2 )



        global_loss.append(loss_local/nb_runs)
        nb+=1

#============Single-task learning for replied task
    #Inputs:
    eta=1e-6
    lamb=1e-1
    eps=1e-3
    nb_runs = 600000
    #Initialization:
    w3=[w for w in sample_floats(0,1,k=32)]
    global_loss=[0,1]
    nb=1
    while abs(global_loss[nb-1]-global_loss[nb])>eps:
        loss_local=0
        for j in tqdm_notebook(range(nb_runs)):
            u = randint(0,len(U)-1)

            target_samples=indx_dict[U[u]]
            target_lables=[Y_train_dict_click[tuple(elem)] for elem in target_samples]
            positive=[target_samples[m] for m in [l for l, e in enumerate(target_lables) if e == 1]]
            negative=[target_samples[m] for m in [l for l, e in enumerate(target_lables) if e == -1]]

            if (len(positive)!=0 and len(negative)!=0): 

                p=randint(0, len(positive)-1)
                n=randint(0, len(negative)-1)

                book_plus_label = Y_train_dict_book[tuple(positive[p])]
                book_minus_label = Y_train_dict_book[tuple(negative[n])]
                reply_plus_label = Y_train_dict_reply[tuple(positive[p])]
                reply_minus_label = Y_train_dict_reply[tuple(negative[n])]

                item_plus = X_train_dict[tuple(positive[p])]
                item_minus = X_train_dict[tuple(negative[n])] 

                diff = ( np.array(item_minus) - np.array(item_plus) ).tolist()            

                l_3_plus = (1./2) * np.log( 1 + ( 1. / np.exp( reply_plus_label * np.dot( [1.] + item_plus , w3 )  ) ) ) 
                l_3_minus = (1./2) * np.log( 1 + ( 1. / np.exp( reply_minus_label * np.dot( [1.] + item_minus , w3 ) ) ) )

                dl_3_plus = ( 1./2) * ( ( (-1.)* reply_plus_label * np.array([1.] + item_plus) ) / ( 1. + np.exp( reply_plus_label * np.dot( [1.] + item_plus , w3 ) ) ) ) 
                dl_3_minus = ( 1./2) * ( ( (-1.)* reply_minus_label * np.array([1.] + item_minus) ) / ( 1. + np.exp( reply_minus_label * np.dot( [1.] + item_minus , w3 ) ) ) )

                if reply_plus_label == 1 :
                    dl_3_plus = 5 * dl_3_plus
                if reply_minus_label == 1 :
                    dl_3_minus = 5 * dl_3_minus


                w3 = w3 - eta * ( (dl_3_plus + dl_3_minus) + (2 * lamb * np.sum( w3 ) ) )    

                loss_local = loss_local + l_3_plus + l_3_minus  + ( lamb * np.linalg.norm( w3,2 )**2 )



        global_loss.append(loss_local/nb_runs)
        nb+=1
#===============================================================
    Y_pred_click={}
    Y_pred_book={}
    Y_pred_reply={}
    for key,value in X_test_dict.iteritems():
        Y_pred_click[key]=sigmoid(np.dot(w1,value))
    for key,value in X_test_dict.iteritems():
        Y_pred_book[key]=sigmoid(np.dot(w2,value))
    for key,value in X_test_dict.iteritems():
        Y_pred_reply[key]=sigmoid(np.dot(w3,value))
        
    for key,value in Y_pred_click.iteritems():
        if value<=0.5:
            Y_pred_click[key]=0
        else:
            Y_pred_click[key]=1
    for key,value in Y_pred_book.iteritems():
        if value<=0.5:
            Y_pred_book[key]=0
        else:
            Y_pred_book[key]=1
    for key,value in Y_pred_reply.iteritems():
        if value<=0.5:
            Y_pred_reply[key]=0
        else:
            Y_pred_reply[key]=1
    
    Y_pred_click_pd=pd.DataFrame(Y_pred_click.items())
    Y_pred_book_pd=pd.DataFrame(Y_pred_book.items())
    Y_pred_reply_pd=pd.DataFrame(Y_pred_reply.items())
    
    Y_pred_click_pd[['user_id','item_id']]=pd.DataFrame(Y_pred_click_pd[0].values.tolist())
    Y_pred_book_pd[['user_id','item_id']]=pd.DataFrame(Y_pred_book_pd[0].values.tolist())
    Y_pred_reply_pd[['user_id','item_id']]=pd.DataFrame(Y_pred_reply_pd[0].values.tolist())
    Y_pred_click_pd = Y_pred_click_pd[["user_id","item_id",1]]
    Y_pred_book_pd = Y_pred_book_pd[["user_id","item_id",1]]
    Y_pred_reply_pd = Y_pred_reply_pd[["user_id","item_id",1]]
    FINAL_click=pd.merge(Y_test_click,Y_pred_click_pd,how='left',on=['user_id','item_id'])
    FINAL_book=pd.merge(Y_test_book,Y_pred_book_pd,how='left',on=['user_id','item_id'])
    FINAL_reply=pd.merge(Y_test_reply,Y_pred_reply_pd,how='left',on=['user_id','item_id'])
    
    print('click')
    print(classification_report(FINAL_click.click, FINAL_click[1]))
    print(confusion_matrix(FINAL_click.click, FINAL_click[1]))
    print('book')
    print(classification_report(FINAL_book.book, FINAL_book[1]))
    print(confusion_matrix(FINAL_book.book, FINAL_book[1]))
    print('reply')
    print(classification_report(FINAL_reply.reply, FINAL_reply[1]))
    print(confusion_matrix(FINAL_reply.reply, FINAL_reply[1]))
    
    false_positive_rate, recall, thresholds = roc_curve(FINAL_click.click, FINAL_click[1])
    roc_auc = auc(false_positive_rate, recall)
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, recall, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.ylabel('Recall')
    plt.xlabel('Fall-out')
    plt.show()
    
    false_positive_rate, recall, thresholds = roc_curve(FINAL_book.book, FINAL_book[1])
    roc_auc = auc(false_positive_rate, recall)
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, recall, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.ylabel('Recall')
    plt.xlabel('Fall-out')
    plt.show()
    
    false_positive_rate, recall, thresholds = roc_curve(FINAL_reply.reply, FINAL_reply[1])
    roc_auc = auc(false_positive_rate, recall)
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, recall, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.ylabel('Recall')
    plt.xlabel('Fall-out')
    plt.show()
    
    scores={}
    for key,value in X_test_dict.iteritems():
        scores[key]=np.dot(value,w1)

    Y_test_click=Y_test_click.set_index(['user_id','item_id'])
    Y_test_dict=Y_test_click.to_dict('index')
    for key,value in scores.iteritems():
        scores[key]=[value]+[Y_test_dict[key]['click']]
    Scores=pd.DataFrame(scores.items())
    Scores[['user_id','item_id']]=pd.DataFrame(Scores[0].values.tolist())
    Scores[['score','Y_test_click']]=pd.DataFrame(Scores[1].values.tolist())
    user_list=np.unique(X_test_std.user_id)
    total_map=[]
    k=1
    for i in tqdm_notebook(range (len(user_list))):
        target_list=Scores[Scores.user_id==user_list[i]].sort_values(by='score',ascending=False)
        up=1
        down=1
        precision=[]
        for j,score in enumerate(target_list.Y_test_click[:k].values.tolist()):
            if score!=0:
                precision.append(float(up)/float(down+j))
                up+=1
            else:
                precision.append(0)
        total_map.append(np.sum(precision[0:k])/k)
    np.mean(total_map)


       
    return w1,w2,w3

In [5]:
normalize("/Users/amirasarbaev/Desktop")